In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers, Model
from keras import ops


@tf.function
def msloss(ytrue, ypred):
  """
  modified loss function with rescaling

  needs some work vor vae
  """
  #ytrue[:] = ytrue[:,tf.where(tf.sum(ytrue[:,:]))]
  #ypred[:] = ypred[:,tf.where(tf.sum(ytrue[:,:]))]

  yte = ytrue[:,:,0]*tf.tanh(ytrue[:,:,1])
  ytp = ytrue[:,:,0]*tf.tanh(ytrue[:,:,2])

  ype = tf.cast(ytrue[:,:,0], tf.float32)*tf.tanh(ypred[:,:,1])
  ypp = tf.cast(ytrue[:,:,0], tf.float32)*tf.tanh(ypred[:,:,2])

  yt = tf.concat([ytrue[:,:,0], yte,ytp], axis=1)
  yp = tf.concat([ypred[:,:,0], ype,ypp], axis=1)

  return tf.reduce_mean(tf.reduce_sum(tf.square(tf.cast(yt, tf.float32)-yp)))


def tmp():
    """For VAE implementation a lot of code is taken from
    https://keras.io/examples/generative/vae/
    """
    import numpy as np
    import tensorflow as tf
    import keras
    from keras import ops
    from keras import layers
    class Sampling(layers.Layer):
      """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

      def __init__(self, **kwargs):
          super().__init__(**kwargs)
          self.seed_generator = keras.random.SeedGenerator()

      def call(self, inputs):
          z_mean, z_log_var = inputs
          batch = ops.shape(z_mean)[0]
          dim = ops.shape(z_mean)[1]
          epsilon = keras.random.normal(shape=(batch, dim))#, seed=self.seed_generator())
          return z_mean + ops.exp(0.5 * z_log_var) * epsilon

    return Sampling

Sampling = tmp()

def dnencoder(vae=0, latent_dim=3):
  encoder_inputs = layers.Input(shape=(19,3,))
  x = layers.BatchNormalization()(encoder_inputs)
  x = layers.Flatten()(x)
  x = layers.Dense(32)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(16)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)

  if vae:
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

  else:
    x = layers.Dense(latent_dim)(x)
    encoder = Model(encoder_inputs, x, name="encoder")

  return encoder

def dndecoder(latent_dim=3):
  decoder_inputs = layers.Input(shape=(latent_dim,))
  x = layers.Dense(16)(decoder_inputs)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(32)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(19*3)(x)
  x = layers.BatchNormalization()(x)
  decoder_outputs = layers.Reshape((19,3))(x)
  return Model(decoder_inputs, decoder_outputs, name="decoder")

def cnencoder(vae=0, latent_dim=8):
  encoder_inputs = layers.Input(shape=(19,3,1))
  x = layers.ZeroPadding2D(padding=(1,0))(encoder_inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(16, kernel_size=(3, 3), strides=1, padding="valid")(x)
  x = layers.ReLU()(x)
  x = layers.AveragePooling2D((3, 1))(x)
  x = layers.Conv2D(32, kernel_size=(3, 1), strides=1, padding="valid")(x)
  x = layers.ReLU()(x)
  x = layers.AveragePooling2D((3, 1))(x)
  x = layers.Flatten()(x)
  if vae:
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
  else:
    x = layers.Dense(latent_dim)(x)
    encoder = Model(encoder_inputs, x, name="encoder")
  return encoder

def cndecoder(latent_dim=8):
  decoder_inputs = layers.Input(shape=(latent_dim,))
  #x = layers.BatchNormalization()
  x = layers.Dense(64)(decoder_inputs)
  x = layers.ReLU()(x)
  x = layers.Reshape((2,1,32))(x)
  x = layers.Conv2D(32,(3, 1), strides=1)(x)
  x = layers.ReLU()(x)
  x = layers.UpSampling2D((3, 1))(x)
  x = layers.ZeroPadding2D(((0,0),(1,1)))(x)
  x = layers.Conv2D(16, (3, 1), strides=1)(x)
  x = layers.ReLU()(x)
  x = layers.UpSampling2D((3, 1))(x)
  x = layers.ZeroPadding2D(((1,0),(0,0)))(x)
  decoder_outputs = layers.Conv2D(1, (3, 3), strides=1)(x)
  return Model(decoder_inputs, decoder_outputs, name="decoder")

In [3]:
@tf.keras.utils.register_keras_serializable()
class AE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def call(self, inputs):
        z = self.encoder(inputs)
        return self.decoder(z)

beta = .6
@tf.keras.utils.register_keras_serializable()
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data[0])
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    msloss(data, reconstruction))
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = (1-beta)*reconstruction_loss + beta*kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [4]:
import h5py
from sklearn.model_selection import train_test_split
from functools import partial

dset = h5py.File('/content/background_for_training.h5', 'r')
dset = {key: dset[key][()] for key in dset.keys()}
"""
Contains keys:
    Particles_Classes : 4 classes of Particles
    Particles_Names : Names of the Particles
    Particles : The data (n, 19,4)
        19 : Indexes are
            - 0 : Missing Transverse Energy
            - 1:4 Up to 4 electrons
            - 4:8 Up to 4 muons
            - 8-18 Up to 10 jets
        Subdimension 4 by idx:
            - 0 : pT (transverse momentum)
            - 1 : eta (pseudorapidity)
            - 2 : phi (azimuthal angle)
            - 3 : class (0=Nothing, 1=Met,2=electron,3=muon,4=jet)
And when something doesnt make sense (ie [:,0,1:4]) its just zero
"""
data = (dset['Particles'])[:,:,:3]
del dset
# Do z score norm to aid in training : They dont specify how they made O(1) : And I assume they mean across all defined objects
#detected_bmap = (data[:,:,3] != 0) # Select defined entries
#mean_pt = tf.reduce_mean(data[detected_bmap, 0])
#std_pt = tf.math.reduce_std(data[detected_bmap, 0])
#data[:,:,0] = ((data[:,:,0] - mean_pt) / std_pt)

sloss = tf.function(partial(msloss))

dn_ae = AE(dnencoder(), dndecoder())
dn_ae.compile(optimizer="Adam", loss=sloss)

#cn_ae = AE(cnencoder(), cndecoder())
#cn_ae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)

dn_vae = VAE(dnencoder(vae=1), dndecoder())
dn_vae.compile(optimizer="Adam", loss=sloss)

#cn_vae = VAE(cnencoder(vae=1), cndecoder())
#cn_vae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)



In [6]:
train, test = train_test_split(data, test_size=0.5)
test, val = train_test_split(test, test_size=0.8)

In [ ]:
dn_ae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
dn_ae.save_weights('dn_ae_weights.weights.h5')
#dn_ae.save('dn_ae_model.keras')

ValueError: You are saving a model that has not yet been built. Try building the model first by calling it on some data or by using `build()`.

In [ ]:
dn_vae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
dn_vae.save_weights('dn_vae_weights.weights.h5')
#dn_ae.save('dn_ae_model.keras')

Epoch 1/100


TypeError: in user code:

    File "<ipython-input-6-8a2447e7bb33>", line 16, in msloss  *
        yte = ytrue[:,:,0]*tf.tanh(ytrue[:,:,1])

    TypeError: tuple indices must be integers or slices, not tuple


In [ ]:
cn_ae = AE(cnencoder(), cndecoder())
cn_ae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)

#cn_vae = VAE(cnencoder(vae=1), cndecoder())
#cn_vae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)

ValueError: Computed output size would be negative. Received `inputs shape=(None, 0, 3, 32)`, `kernel shape=(3, 1, 32, 16)`, `dilation_rate=[1 1]`.

## my plotting didn't work so using felix's for now

In [10]:
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import h5py
dn_ae = AE(dnencoder(), dndecoder())
dn_ae.compile(optimizer="Adam", loss=sloss)
dn_ae.fit(train[:2], train[:2], batch_size=1)
dn_ae.load_weights('/content/dn_ae_weights.weights.h5')

# For ROC we also need a background set lets use 1e5 background events
background = h5py.File('/content/background_for_training.h5', 'r')['Particles'][:,:,:3]#[:1000000,:,:3]
# ================================
# PLOT OUTLINE BASED ON THE PAPER
# ================================
def compute_io_score(y_true, y_pred):
    """
    Compute input-output (IO) anomaly score based on reconstruction loss (MSE).
    Used for: AE and VAE models

    Args:
        y_true (np.array): True input data [batch_size, height, width, channels]
        y_pred (np.array): Reconstructed output from the model [batch_size, height, width, channels]

    Returns:
        np.array: Anomaly score for each sample based on MSE
    """
    return np.square(y_true - y_pred)
# === FIGURE II: ROC Curves for CNN and DNN Models ===
# Data:
# - x-axis → False Positive Rate (FPR)
# - y-axis → True Positive Rate (TPR)
# - Models:
#   - CNN VAE (using IO, KL divergence, Rz scores)
#   - DNN VAE (using IO, KL divergence, Rz scores)
#   - CNN AE (using IO score)
#   - DNN AE (using IO score)
# Benchmark Models:
# - LQ → bτ
# - A → 4ℓ
# Method:
# - Compute ROC curves based on anomaly scores:
#   - MSE for AEs
#   - KL and Rz for VAEs
#   - Plot separate ROC curves for CNN and DNN
def figureII():
    def do_roc(ax, score, y_true, text):
        fpr, tpr, _ = roc_curve(y_true, score)
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, label=f'{text} (AUC = {roc_auc:.3f})')
    fig, ax = plt.subplots(2, 2, figsize=(16, 16))
    ax = ax.flatten()

    # CNN LQ -> bτ
    #Load SIGNAL ONLY Data
    dset = h5py.File('/content/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    #Concatenate dset and background - Load as much bkground as data
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    #dset[:,:,0] = ((dset[:,:,0] - mean_pt) / std_pt)

  #  CNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_vae_beta1.0.keras')
  #  CNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_ae.keras')

   # cnn_io_vae = compute_io_score(dset, CNN_VAE.predict(dset)[:,:,:,0])
    #do_roc(ax[0], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

   # cnn_dkl_vae = compute_kl_score(*CNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[0], cnn_dkl_vae, labels, "VAE D_{KL}")

   # cnn_Rz_vae = compute_rz_score(*CNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[0], cnn_Rz_vae, labels, "VAE R_Z")

   # cnn_io_ae = compute_io_score(dset, CNN_AE.predict(dset)[:,:,:,0])
    #do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "leptoquark")#"IO AE")

    dset = h5py.File('/content/hChToTauNu_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    #Concatenate dset and background - Load as much bkground as data
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "hChToTauNu")#"IO AE")

    dset = h5py.File('/content/Ato4l_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])
    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "Ato4l")#"IO AE")

    dset = h5py.File('/content/hToTauTau_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "hToTauTau")

    # DNN LQ -> bτ
    #DNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_vae_beta1.0.keras')
    #DNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_ae.keras')
    #dnn_ae.load_weights('/content/dnn.weights.h5')


#    dnn_io_vae = compute_io_score(dset, DNN_VAE.predict(dset))
 #   do_roc(ax[1], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

  #  dnn_dkl_vae = compute_kl_score(*DNN_VAE.encoder.predict(dset)[0:2])
   # do_roc(ax[1], dnn_dkl_vae, labels, "VAE D_{KL}")

   # dnn_Rz_vae = compute_rz_score(*DNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[1], dnn_Rz_vae, labels, "VAE R_Z")

    #dnn_io_ae = msloss(dset, dn_ae.predict(dset))
    #do_roc(ax[1], tf.reduce_mean(dnn_io_ae), labels, "IO AE")

    """
    # CNN A-> 4l
    dset = h5py.File('Ato4l_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    cnn_io_vae = compute_io_score(dset, CNN_VAE.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*CNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*CNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, CNN_AE.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # DNN A-> 4l
    dnn_io_vae = compute_io_score(dset, DNN_VAE.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*DNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*DNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[3], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")
"""


    # And formating
    legend_text = [
        "CNN ROC LQ → bτ",
        "DNN ROC LQ → bτ",
        "CNN ROC A → 4ℓ",
        "DNN ROC A → 4ℓ"
    ]
    for i, a in enumerate(ax):
        a.set_xlim(1e-6, 1)
        a.set_ylim(1e-6, 1)
        a.set_xscale('log')
        a.set_yscale('log')
        a.set_xlabel('False Positive Rate')
        a.set_ylabel('True Positive Rate')
        a.set_xticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.set_yticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.grid(False)
        a.plot([1e-6, 1], [1e-6, 1], color='grey', linestyle='--', alpha=0.4)
        a.axvline(x=1e-5, color='red', linestyle='--', label='Label')
        # Add Legend Handle
        handles, labels = a.get_legend_handles_labels()
        # Create the custom text as a Line2D object
        #extra = Line2D([0], [0], color='none', label='')
        # Insert at the beginning of the list
        #handles.insert(0, extra)
        #labels.insert(0, legend_text[i])
        a.legend(handles=handles, labels=labels, loc='lower right')

    plt.show()

figureII()



2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 8816.0928


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 50 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 4 objects could not be loaded. Example error message for object <BatchNormalization name=batch_normalization_41, built=True>:

Layer 'batch_normalization_41' expected 4 variables, but received 0 variables during loading. Expected: ['gamma', 'beta', 'moving_mean', 'moving_variance']

List of objects that could not be loaded:
[<BatchNormalization name=batch_normalization_41, built=True>, <BatchNormalization name=batch_normalization_36, built=True>, <BatchNormalization name=batch_normalization_37, built=True>, <BatchNormalization name=batch_normalization_38, built=True>]